In [3]:
from PIL import Image
import os

input_folder = 'D:/desk/garden/image'  # 图像文件夹路径
output_folder = 'D:/desk/garden/image'
new_size = (1280, 720) # 设置新的大小

# 遍历文件夹中的所有图像并resize
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(input_folder, filename)
        img = Image.open(img_path)
        img = img.resize(new_size)
        folder_path = os.path.join(output_folder, filename)
        img.save(folder_path)

In [2]:
import os
import random
import pandas as pd
from tkinter import *
from tkinter import ttk
from PIL import Image, ImageTk

folder_path = 'E:/Dataset/GNN_Perception/SVI/scoring'  # 图像文件夹路径
        
num_pairs = 1000  # 每轮需要比较的图像对数（输入文件夹中所有的图像数量）
num_rounds = 1  # 总共需要比较的轮）
supported_extensions = ['.jpg', '.jpeg', '.png']  # 支持的图像文件扩展名

# 获取所有支持的图片文件列表
images = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and
          os.path.splitext(f)[1].lower() in supported_extensions]

# 创建空字典，用于记录每张图片被选择的次数
# selections = {}
# for img in images:
#     selections[img] = 0
# 创建空字典，用于记录每张图片被选择的次数
selections = {}
for img in images:
    selections[img] = 0
  # 添加相等选项计数器


class Application(Frame):
    def __init__(self, master=None):
        try:
            super().__init__(master)
            self.master = master
            self.master.protocol("WM_DELETE_WINDOW", self.on_close)
            self.pack()

            # 初始化变量
            self.round_index = 0
            self.pair_index = 0

            # 生成图片对
            self.generate_pairs()

            # 显示第一轮比较任务
            self.show_images()
        except Exception as e:
            print('An error occurred:', e)
            self.save_scores()
            self.master.destroy()

    def show_images(self):
        global pairs
        if self.round_index >= num_rounds:
            # 所有比较完成，保存结果并退出
            self.save_scores()
            self.master.destroy()
            return

        if self.pair_index >= num_pairs:
            # 一轮比较完成，保存结果并重新生成新的任务对
            self.save_scores()
            self.generate_pairs()
            self.pair_index = 0
            self.round_index += 1

        # 获取当前对比的两张图像
        pair = pairs[self.pair_index]
        if len(pair) != 2:
            # 图像对不完整，退出程序
            self.save_scores()
            self.master.destroy()
            return

        img1 = Image.open(os.path.join(folder_path, pair[0]))
        img2 = Image.open(os.path.join(folder_path, pair[1]))
        img1 = img1.resize((img1.width // 3, img1.height // 3))  #调整展示图像的尺寸
        img2 = img2.resize((img2.width // 3, img2.height // 3))  #调整展示图像的尺寸

        # 添加文本
        text_label = Label(self, text="Q4：在这个地方，我的注意力被许多有趣的事情所吸引",font=("Microsoft YaHei UI", 12))
        text_label.grid(row=5, column=0, columnspan=5)
        
        # # 添加进度条
        # self.progress_bar = ttk.Progressbar(self, orient=HORIZONTAL, length=200, mode='determinate')
        # self.progress_bar.grid(row=8, column=0, columnspan=2)
        # self.progress_bar['maximum'] = num_rounds * num_pairs
        
        # self.progress_bar['value'] = self.round_index * num_pairs + self.pair_index + 1
        # self.progress_bar.update_idletasks()   # 使用 update_idletasks() 方法更新进度条


        # 显示图像
        tkimg1 = ImageTk.PhotoImage(img1)
        tkimg2 = ImageTk.PhotoImage(img2)
        self.label1 = Label(self, image=tkimg1)
        self.label1.image = tkimg1
        self.label1.grid(row=0, column=0)
        self.label2 = Label(self, image=tkimg2)
        self.label2.image = tkimg2
        self.label2.grid(row=0, column=1)

        # 添加选择按钮
        self.button1 = Button(self, text='选择左边的图片', command=lambda: self.choose_image(pair[0]))
        self.button1.grid(row=1, column=0)
        self.button2 = Button(self, text='选择右边的图片', command=lambda: self.choose_image(pair[1]))
        self.button2.grid(row=1, column=1)
        # 添加同时选择两张图片的按钮
        self.button3 = Button(self, text='全部', command=lambda: self.choose_image(pair[0], pair[1]))
        self.button3.grid(row=1, column=0, columnspan=2)
        # 添加都不选择的按钮
        self.button4 = Button(self, text='都不', command=lambda: self.choose_image(None))
        self.button4.grid(row=2, column=0, columnspan=2)

    def generate_pairs(self):
        global pairs
        pairs = []
        while len(pairs) < num_pairs:
            pair = random.sample(images, 2)
            if pair not in pairs:
                pairs.append(pair)

    def choose_image(self, img1, img2=None):
        # 增加当前图片的被选择次数
        # 增加当前图片的被选择次数
        if img1 is not None:
            selections[img1] += 1
        if img2 is not None:
            selections[img2] += 1

        # 显示下一对图像
        self.pair_index += 1
        self.label1.destroy()
        self.label2.destroy()
        self.button1.destroy()
        self.button2.destroy()
        self.button3.destroy()
        self.show_images()

    def save_scores(self):
        scores = []
        for img in images:
            scores.append(selections[img])
        # Add count for equal option
        # scores.append(selections['equal'])
        # scores.append(selections['none'])
        df = pd.DataFrame({'image': images, 'score': scores})
        # df = pd.DataFrame({'image': images, 'score': scores})
        # df.to_csv('E:/Dataset/GNN_Perception/SVI/score_Q1_4.csv', index=False)
        return df

    def on_close(self):
        try:
            self.save_scores()
            self.master.destroy()
        except Exception as e:
            print('An error occurred:', e)
            self.master.destroy()

# 创建 GUI 程序并运行
root = Tk()
app = Application(master=root)
app.mainloop()

In [5]:
app.save_scores().head()

,image,score
0,114.156143869_30.6128186974.png,0
1,114.158411111_30.6206021664.png,0
2,114.15942885_30.574736697.png,0
3,114.159630471_30.5788771409.png,1
4,114.15989869_30.6073719864.png,3


In [3]:
import pandas as pd
from trueskill import Rating, rate_1vs1

# 读取图片得分数据
df = app.save_scores()

# 初始化等级分
ratings = {}
for img in df['image']:
    ratings[img] = Rating()

# 计算每张图片的等级分
for i, row in df.iterrows():
    img1 = row['image']
    score1 = row['score']
    r1 = ratings[img1]
    for img2 in ratings.keys():
        if img2 == img1:
            continue
        r2 = ratings[img2]
        score2 = df.loc[df['image'] == img2]['score'].item()
        if score1 > score2:
            r1, r2 = rate_1vs1(r1, r2)
        elif score1 < score2:
            r2, r1 = rate_1vs1(r2, r1)
        ratings[img1], ratings[img2] = r1, r2    
        
# 映射等级分到 0~10 的浮点值
min_val = min([r.mu - 3 * r.sigma for r in ratings.values()])
max_val = max([r.mu + 3 * r.sigma for r in ratings.values()])
for img, r in ratings.items():
    val = (r.mu - min_val) / (max_val - min_val) * 10   #调整输出值的区间0-10
    df.loc[df['image'] == img, 'score'] = val

# 打印每张图片的归一化分数值
#print(df)
    
# 保存到 CSV 文件中
df.to_csv('E:/Dataset/GNN_Perception/Graph/score_Q4.csv', index=False)

In [26]:
df.head()

,image,score
0,nh1.jpg,2.957863
1,nh101.jpg,2.966491
2,nh106.jpg,1.653888
3,nh11.jpg,2.428644
4,nh111.jpg,0.483444
